### UNBALANCED DATASET 
- https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set  
- http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/  
- https://github.com/scikit-learn-contrib/imbalanced-learn  
- https://shiring.github.io/machine_learning/2017/04/02/unbalanced  


#### TRY small equal subset of ALL DATA  
N=2000 each : a bit above chance model v0/v1 (acc~55-61%) || Not great ...  ref -- model_v1#.h5  
Ref: http://localhost:8888/notebooks/load_Imgs_runtestCNN_v0.ipynb


#### > TRY ALL DATA  -- 1:10 ratio train-test   
not great -- zero recall for images with buildings... ref -- model_v2#.h5   
Ref: http://localhost:8888/notebooks/load_Imgs_runtestCNN_v1.ipynb


#### >> TRY Upsampling Bclass and Downsampling NBclass
N=6000 each : PROMISING with cnn_model v3 (train-test acc~97%) | transfer_VGG16_model v4 (train-test acc~93%)   
Ref: http://localhost:8888/notebooks/load_Imgs_runtestCNN_v1.ipynb   ||  http://localhost:8888/notebooks/load_Imgs_runtestCNN_v2_tryVGG16Transfer.ipynb
- train-test: N=6000 each --- b_2000 + 4000_randomSampling || nb_6000   
- ** however val data also has mixture of oversampled data... so not ideal...



#### General REQs

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

In [2]:
# !pwd
# path='/Users/hrm/Documents/Dropbox/DSrelated/DG_datachallenge'
path = '/home/ubuntu/pynb/DG_datachallenge'

#### File paths

In [3]:
## Get fullpaths to img files
import glob, os, sys

btmp = glob.glob(os.path.join(path+'/building_set/buildings/','*.tif')) #os.listdir( path+'/building_set/buildings/' )
# buildingFileNames
nbtmp = glob.glob(os.path.join(path+'/building_set/no_buildings/','*.tif')) #os.listdir( path+'/building_set/no_buildings' )
# nobuildingFileNames

buildingFileNames = btmp
nobuildingFileNames = nbtmp

In [4]:
btmp[:3]

['/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/289840.tif',
 '/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/188789.tif',
 '/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/641926.tif']

In [5]:
len(btmp) , len(nbtmp)

(2000, 20000)

#### Define traintest data

In [4]:
Nimg = 6000

btmp1 = [btmp[i] for i in np.random.choice(len(btmp),Nimg-2000,replace=True).tolist()]
nbtmp1 = [nbtmp[i] for i in np.random.choice(len(nbtmp),Nimg,replace=False).tolist()]

In [5]:
# btmp1.extend(btmp)
len(btmp1)

4000

In [6]:
btmp1.extend(btmp)
buildingFileNames = btmp1
nobuildingFileNames = nbtmp1

In [7]:
len(buildingFileNames)

6000

#### combine all traintest data

In [8]:
buildDF = pd.DataFrame(buildingFileNames, columns=['filename'])
buildDF['Blabel'] = 1
nobuildDF = pd.DataFrame(nobuildingFileNames, columns=['filename'])
nobuildDF['Blabel'] = 0

FileIdx_all = pd.concat([buildDF,nobuildDF])

In [9]:
FileIdx_all.Blabel.value_counts()

1    6000
0    6000
Name: Blabel, dtype: int64

#### Define basic image preprocessing 

In [10]:
import numpy as np
from skimage import color, exposure, transform

# NUM_CLASSES = 2
IMG_SIZE = 256

def preprocess_img(img):
    img=np.array(img)
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE), mode='constant')

    # roll color axis to axis 0
    #img = np.rollaxis(img,-1)

    return img

#### Define function to load & run image processing --> image np_arrays

In [11]:
from skimage import io
import os
import glob

def get_Fileclass(img_path):
    return str(img_path.split('/')[-2])

def get_FileName(img_path):
    return str(img_path.split('/')[-1])



def makeImageXYvars(FileIdx_file):

    imgs = []
    labels = []
    
    cnt = 0
    N = 0 
    imgN = 500
    

    # TrainImgPaths = FileIdx_Train.ix[FileIdx_Train.sample(2000).Blabel.index.tolist()]
    # FileImgPaths = FileIdx_file.ix[FileIdx_file.sample(2000).Blabel.index.tolist()] #randomly samples each time...
    
    FileImgPaths = FileIdx_file

    # for img_path in TrainImgPaths.filename:
    for img_path in FileImgPaths.filename:    

        img = preprocess_img(io.imread(img_path))  ## add image more pre-processing later...

        label = get_Fileclass(img_path)
        #print('preprocessing image : '+ label + ' -- ' + get_FileName(img_path))

        imgs.append(img)
        labels.append(label)

        if cnt%imgN==0:
            if N==0 & cnt==0:
                print('start processing....')
            else:
                print('preprocessed {0} images ...'.format((N*imgN)))
            N += 1
        
        cnt += 1


    X = np.array(imgs, dtype='float32')

    # Make one hot targets
    Y= FileImgPaths.Blabel.values 

    return X, Y, FileImgPaths


#### Run image processing --> np_arrays

In [12]:
# X,Y, FileImgPaths = makeImageXYvars(FileIdx_Train)
# X,Y, FileImgPaths = makeImageXYvars(FileIdx_Test)

X,Y, FileImgPaths = makeImageXYvars(FileIdx_all)

start processing....
preprocessed 500 images ...
preprocessed 1000 images ...
preprocessed 1500 images ...
preprocessed 2000 images ...
preprocessed 2500 images ...
preprocessed 3000 images ...
preprocessed 3500 images ...
preprocessed 4000 images ...
preprocessed 4500 images ...
preprocessed 5000 images ...
preprocessed 5500 images ...
preprocessed 6000 images ...
preprocessed 6500 images ...
preprocessed 7000 images ...
preprocessed 7500 images ...
preprocessed 8000 images ...
preprocessed 8500 images ...
preprocessed 9000 images ...
preprocessed 9500 images ...
preprocessed 10000 images ...
preprocessed 10500 images ...
preprocessed 11000 images ...
preprocessed 11500 images ...


In [13]:
Y.shape, Y
# ((4000,), array([1, 0, 1, ..., 0, 1, 0]))

((12000,), array([1, 1, 1, ..., 0, 0, 0]))

In [14]:
### RESHAPE Y
from keras.utils import np_utils
Y=np_utils.to_categorical(Y, 2)

Using TensorFlow backend.


In [17]:
### Include all images
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, 
                                                test_size=0.2, random_state=4859, 
                                                stratify=Y)

In [18]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((9600, 256, 256, 3), (2400, 256, 256, 3), (9600, 2), (2400, 2))

In [40]:
Xtest

array([[[[ 0.14533462,  0.08018462,  0.05011538],
         [ 0.18672346,  0.12199686,  0.09823303],
         [ 0.27343839,  0.20749795,  0.17159176],
         ..., 
         [ 0.3326306 ,  0.25188801,  0.2357395 ],
         [ 0.39754024,  0.30726337,  0.28920802],
         [ 0.40747264,  0.31692314,  0.29881325]],

        [[ 0.27411598,  0.17979057,  0.13625576],
         [ 0.19345741,  0.12695293,  0.091363  ],
         [ 0.15564919,  0.10765772,  0.08105905],
         ..., 
         [ 0.52691501,  0.42058843,  0.39932311],
         [ 0.43902066,  0.34304214,  0.32384643],
         [ 0.30421129,  0.22670396,  0.21120249]],

        [[ 0.29901132,  0.19919762,  0.15312976],
         [ 0.20316121,  0.13532358,  0.0986192 ],
         [ 0.17929889,  0.12657036,  0.09731615],
         ..., 
         [ 0.52762759,  0.42127022,  0.39999875],
         [ 0.33178923,  0.25096816,  0.23480393],
         [ 0.12877189,  0.08149989,  0.07204549]],

        ..., 
        [[ 0.88785416,  0.77442509,

In [21]:
FileImgPaths.filename.value_counts()
# FileImgPaths.filename.values[:10]

/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/70302.tif        9
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/647646.tif       9
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/79048.tif        8
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/678214.tif       8
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/704938.tif       8
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/710264.tif       8
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/204578.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/112846.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/230397.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/679040.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/614487.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/703706.tif       7
/home/ubuntu/pynb/DG_datachallenge/building_set/buildings/99210.tif        7

#### SAVE FILES

In [20]:
### SAVE FILES

import pickle
import numpy as np

# np.save(path+'/building_set/TrainX_v0',X)
# np.save(path+'/building_set/TrainY_v0',Y)

# np.save(path+'/building_set/TestX_v0',X)
# np.save(path+'/building_set/TestY_v0',Y)



## LARGE files....
# np.save(path+'/building_set/X_all',X)
# np.save(path+'/building_set/Y_all',Y)

np.save(path+'/building_set/X_ubdnb6000_all',X)
np.save(path+'/building_set/Y_ubdnb6000_all',Y)
FileImgPaths.to_csv(path+'/building_set/XY_ubdnb6000_all.csv')



#### LOAD FILES

In [ ]:
# LOAD saved vars


# f = np.load('filepath.npy')
# f.shape


In [95]:
import keras
keras.backend.image_data_format()

'channels_last'